In [1]:
import torch
from torch.utils.data import DataLoader
from miditok import REMI
from miditok.pytorch_data import DatasetMIDI, DataCollator
from pathlib import Path
import json
import pandas as pd
import numpy as np
import random
import utils
import importlib

In [2]:
importlib.reload(utils)

<module 'utils' from '/mnt/nfs_share_magnet1/lafuente/symbolic_music/author-profiling/experiments/e17/utils.py'>

In [3]:
seed = 42
if seed is not None:
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [4]:
k=1
train_data=pd.read_csv(f'../../train data/k folds/type0_train_set_k{k}.csv')
validation_data=pd.read_csv(f'../../train data/k folds/type0_test_set_k{k}.csv')

## 1. Get features vector

### 1.1 Create dataloader

In [5]:
val_dataloader=utils.create_data_loader(scores_df=validation_data,paths_column_name='type0_path')
train_dataloader=utils.create_data_loader(scores_df=train_data,paths_column_name='type0_path')

/mnt/nfs_share_magnet1/lafuente/miniconda3/envs/symbolic_music/lib/python3.12/site-packages/miditok/midi_tokenizer.py:3252: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  self.config = TokenizerConfig()
/mnt/nfs_share_magnet1/lafuente/miniconda3/envs/symbolic_music/lib/python3.12/site-packages/miditok/classes.py:702: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  return cls(**input_dict, **kwargs)
/mnt/nfs_share_magnet1/lafuente/miniconda3/envs/symbolic_music/lib/python3.12/site-packages/miditok/classes.py:702: UserWarning: Argument nb_tempos has been renamed num_tempos, you should consider to updateyour code with this new argument name.
  return cls(**input_dict, **kwargs)


### 1.2 Get feature vectors from pre-trained model

In [6]:
train_feature_vectors=utils.get_feature_vectors(dataloader=train_dataloader,
                                                dataframe=train_data,
                                                set_type='train', 
                                                feature_tensors=False)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
computing feature tensors: 100%|██████████████| 184/184 [10:07<00:00,  3.30s/it]


In [7]:
val_feature_vectors=utils.get_feature_vectors(dataloader=val_dataloader,
                                                dataframe=validation_data,
                                                set_type='val', 
                                                feature_tensors=False)

computing feature tensors: 100%|████████████████| 46/46 [02:50<00:00,  3.71s/it]


## 2. Train MLP

In [9]:
predictions_df_train_e17,metrics_df,predictions_df_val_e17=utils.train_MLP_classfier(train_feature_vectors=train_feature_vectors,
                          val_feature_vectors=val_feature_vectors,
                          k=k)

MLP(
  (fc1): Linear(in_features=523264, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=2, bias=True)
  (relu): ReLU()
)
Epoch 0 | Train Loss   5.20| Train Balanced Accuracy   0.56
Epoch 0 | Validation Loss   3.07| Validation Balanced Accuracy:   0.51
Epoch 1 | Train Loss   0.95| Train Balanced Accuracy   0.79
Epoch 1 | Validation Loss   5.71| Validation Balanced Accuracy:   0.50
Epoch 2 | Train Loss   0.31| Train Balanced Accuracy   0.90
Epoch 2 | Validation Loss   1.41| Validation Balanced Accuracy:   0.54
Epoch 3 | Train Loss   0.03| Train Balanced Accuracy   0.99
Epoch 3 | Validation Loss   1.42| Validation Balanced Accuracy:   0.53
Epoch 4 | Train Loss   0.02| Train Balanced Accuracy   1.00
Epoch 4 | Validation Loss   1.41| Validation Balanced Accuracy:   0.52
Epoch 5 | Train Loss   0.01| Train Balanced Accuracy   1.00
Epoch 5 | Valida

In [ ]:
predictions_df_train_e17.to_csv(f'predictions_df_train_e17_k{k}_full_seqs.csv')
metrics_df.to_csv(f'metrics_df_e17_k{k}_full_seqs.csv',index=False)
predictions_df_val_e17.to_csv(f'predictions_df_test_e17_k{k}_full_seqs.csv')

In [ ]:
##============== k-folds data ===================##
#full_df=pd.DataFrame([])

#for k in range(1,6):

#    predictions=pd.read_csv(f'predictions_df_test_e17_k{k}.csv')

#    test_set=pd.read_csv(f'../../train data/k folds/type0_test_set_k{k}.csv')

#    temp_df=pd.concat([predictions,test_set],axis=1)

 #   full_df=pd.concat([full_df,temp_df],axis=0)

#full_df.drop(columns='Unnamed: 0').to_csv('../../dataframes/cross_validation_probabilities.csv',index=False)